In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.interpolate import LinearNDInterpolator
import os

from astropy.table import Table
import astropy.units as u

from snewpy import model_path
from snewpy.neutrino import Flavor
from snewpy.models.ccsn import Analytic3Species

from asteria.simulation import Simulation
from asteria import set_rcparams


from snewpy.models.base import SupernovaModel
from snewpy.neutrino import Flavor
from snewpy.models.ccsn import Mori_2023
import snewpy

from glob import glob
from scipy.interpolate import PchipInterpolator
import sys


import snewpy.models.ccsn as ccsn
import scipy.stats as st
from astropy.table import Table, Column, vstack
from astropy.io import *

set_rcparams()
from cycler import cycler
import emcee
import corner
from matplotlib import rcParams
mpl.rcParams['axes.prop_cycle'] = cycler("color", ['#ff355e', '#fd5b78', '#ff6037', '#ff9966', '#ff9933', '#ffcc33', '#ffff66', '#ccff00', '#66ff66', '#aaf0d1', '#50bfe6', '#16d0cb', '#6a1b9a', '#ff00cc' ])
from math import lgamma
from scipy.special import logsumexp

In [2]:
dist=30 * u.kpc
Elow=0*u.MeV
Ehigh=100*u.MeV
deltE=1*u.MeV
Tlow=-0.29521*u.s
Thigh=0.29479*u.s
deltT=1*u.ms
mix_scheme='AdiabaticMSW'
hierarch='normal'

mod= {
    'name': 'Mori_2023',
    'param': {
        'axion_mass':200*u.MeV, 'axion_coupling':4*1e-10/u.GeV
    }
}
sim= Simulation(model=mod,
                 distance=dist, 
                 Emin=Elow, Emax=Ehigh, dE=deltE,
                 tmin=Tlow, tmax=Thigh,  dt=deltT,
                 mixing_scheme=mix_scheme,
                 hierarchy=hierarch)
sim.run()

In [3]:
dt=10*u.ms
t, hits = sim.detector_hits(dt)
t, signal = sim.detector_signal(dt)
i3b = sim.detector.i3_bg(dt, size=hits.size)
dcb = sim.detector.dc_bg(dt, size=hits.size)
bkg=i3b+dcb
bkg_mean=np.mean(bkg)
tot=bkg+hits
tab=Table(data=[t,signal],names=['time','signal'])
tab.write('newtimesignal,10kpc,10ms,200,4.fits')

In [1]:
#to make the data file, ma is set to 100, ga is set to 10, and tot (bkg+hits) is used for the data